# Hyperparameter Tuning using HyperDrive

Install and import dependencies

In [1]:
!pip install kaggle

     |████████████████████████████████| 58 kB 3.5 MB/s eta 0:00:011
     |████████████████████████████████| 78 kB 4.2 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.9-py3-none-any.whl size=73265 sha256=c0d5dcbc8ba1960c7b1bcfd51802bf2b3c13dc71423556851bde4b60bf9f56ea
  Stored in directory: /home/azureuser/.cache/pip/wheels/9d/50/3d/2644504bb1e8c782f3fef5984f03d76fc4a74698fdec128b29
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6769 sha256=dcea7a1e266a0be02df60e2cbd6c288ef77a385cc550888969b1952f251c41ca
  Stored in directory: /home/azureuser/.cache/pip/wheels/72/e6/db/122611605e60148f54ee2abaca98b2bbeafc6e22486a867bad
  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1909 sha256=ab4507dff7efaa168c0a8cba330c0dae52d8330db95857ee59c6ea521aee7afd
  Stored in directory: /home/azureuser/.cache/pip/wheels/7d/51/b0/c584cbdd0a8fc685d68677e58cde93814cbbc7fd9867fb5fe6
Successfully built kaggle python-slugify sl

If the import of data_prep fails, see the README on how to download and copy the kaggle.json

In [3]:
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Model, Webservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import joblib

import data_prep

## Dataset

In [4]:
ws = Workspace.from_config()
ws.write_config(path='./training')
experiment_name = 'hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)

dataset = data_prep.get_dataset(ws)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FJ234ZKRL to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/35e1f6d4-6a27-4b46-8e5d-322e3ef3dbda/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


## Find or Create Compute Cluster

In [5]:
cpu_cluster_name = "cpu-cluster"

# Check if the cluster exists. If there is an error, create the cluster
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                            max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

In the custom training script, we define a Scikit Learn Random Forrest Classifier. There are 4 hyperparameters we are going to tune with Hyperdrive: 
* n_estimators: the number of trees in the Random forrest
* max_depth: the maximum depth of the trees in the forrest
* min_samples_split: the minimum number of samples required to split an internal node
* min_samples_leaf: the minimum number of samples required to be at a leaf node

In [6]:
# Early termination policy
early_termination_policy = BanditPolicy(slack_amount=0.2)

# Random Parameter sampling
param_sampling = RandomParameterSampling({
                    "n_estimators": choice(10,50,100,150,200),
                    "max_depth": choice(0,2,5,10),
                    "min_samples_split": choice(2,3,4,5),
                    "min_samples_leaf": choice(1,2,3,4,5)
                })

# SKLearn Estimator
estimator = SKLearn(source_directory="./training", 
                entry_script="train.py",
                compute_target=cpu_cluster)

# Hyperdrive Configuration
hyperdrive_run_config = HyperDriveConfig(param_sampling,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    estimator=estimator,
                                    policy=early_termination_policy)

In [7]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


## Run Details

In [8]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Num Estimators:',parameter_values[7])
print('\n Max Depth:',parameter_values[1])
print('\n Min Samples Split:',parameter_values[5])
print('\n Min Samples Leaf:',parameter_values[3])

Best Run Id:  HD_b26beb43-d9f2-4ff6-a39d-439d40f24e1a_37

 Accuracy: 0.9996926859250154

 Num Estimators: 100

 Max Depth: 0

 Min Samples Split: 2

 Min Samples Leaf: 1


In [11]:
model = best_run.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib', model_framework=Model.Framework.SCIKITLEARN)

In [17]:
print(Model.get_model_path('best_hyperdrive_model', version=1, _workspace=ws))

azureml-models/best_hyperdrive_model/1/model.joblib


## Model Deployment

Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model and deploy the model as a web service.

In [19]:
rest_service = Model.deploy(ws, "best-model-service", [model])

rest_service.wait_for_deployment(show_output=True)
rest_service.update(enable_app_insights=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


Send a request to the web service you deployed to test it.

In [20]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [12]:
import json

data = {"data": 
          [ 
            { 
              "Name": "memtest.exe", 
              "md5": "631ea355665f28d4707448e442fbf5b8", 
              "Machine": 332, 
              "SizeOfOptionalHeader": 224, 
              "Characteristics": 258, 
              "MajorLinkerVersion": 9, 
              "MinorLinkerVersion": 0, 
              "SizeOfCode": 361984, 
              "SizeOfInitializedData": 115712, 
              "SizeOfUninitializedData": 0, 
              "AddressOfEntryPoint": 6135, 
              "BaseOfCode": 4096, 
              "BaseOfData": 372736, 
              "ImageBase": 4194304, 
              "SectionAlignment": 4096, 
              "FileAlignment": 512, 
              "MajorOperatingSystemVersion": 0, 
              "MinorOperatingSystemVersion": 0, 
              "MajorImageVersion": 0, 
              "MinorImageVersion": 0, 
              "MajorSubsystemVersion": 1, 
              "MinorSubsystemVersion": 0, 
              "SizeOfImage": 1036288, 
              "SizeOfHeaders": 1024, 
              "CheckSum": 485887, 
              "Subsystem": 16, 
              "DllCharacteristics": 1024, 
              "SizeOfStackReserve": 1048576, 
              "SizeOfStackCommit": 4096, 
              "SizeOfHeapReserve": 1048576, 
              "SizeOfHeapCommit": 4096, 
              "LoaderFlags": 0, 
              "NumberOfRvaAndSizes": 16, 
              "SectionsNb": 8, 
              "SectionsMeanEntropy": 5.7668065537, 
              "SectionsMinEntropy": 3.6074295755, 
              "SectionsMaxEntropy": 7.2210507289, 
              "SectionsMeanRawsize": 59712, 
              "SectionsMinRawsize": 1024, 
              "SectionMaxRawsize": 325120, 
              "SectionsMeanVirtualsize": 126875.875, 
              "SectionsMinVirtualsize": 896, 
              "SectionMaxVirtualsize": 551848, 
              "ImportsNbDLL": 0, 
              "ImportsNb": 0, 
              "ImportsNbOrdinal": 0, 
              "ExportNb": 0, 
              "ResourcesNb": 4, 
              "ResourcesMeanEntropy": 3.262822711, 
              "ResourcesMinEntropy": 2.5688438236, 
              "ResourcesMaxEntropy": 3.5379393642, 
              "ResourcesMeanSize": 8797, 
              "ResourcesMinSize": 216, 
              "ResourcesMaxSize": 18032, 
              "LoadConfigurationSize": 0, 
              "VersionInformationSize": 16 
            } 
          ]
        }
  
# Convert to JSON string
input_data = json.dumps(data)

In [ ]:
import requests

response = requests.post(rest_service, 
                         headers=auth_header, 
                         json=input_data
                        )

print(response.json())

In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = rest_service.get_logs()

for line in logs.split('\n'):
    print(line)